In [1]:
from __future__ import print_function
import numpy as np
import torch.nn.functional as nnFunctions
from torch.autograd import Variable
import torch
from torch import nn
import torchvision
from PIL import Image
import skimage.io as io
import pickle
import sys

In [2]:
class myConvNet(nn.Module):
    #constructor
    def __init__(self):
        super(myConvNet, self).__init__()
        #defining layers in convnet
        #input size=1*657*1625
        self.conv1 = nn.Conv2d(512,512, kernel_size=3,stride=1,padding=1)
        self.conv2 = nn.Conv2d(512,256, kernel_size=3,stride=1,padding=1)
        #self.bn1=nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1)
        self.pconv1= nn.Conv2d(256,128, kernel_size=(3,3),stride=1,padding=(1,1))
        #self.bn2=nn.BatchNorm2d(64)
        self.pconv2= nn.Conv2d(256,128, kernel_size=(3,7),stride=1,padding=(1,3))
        self.pconv3= nn.Conv2d(256,128, kernel_size=(7,3),stride=1,padding=(3,1))
        
        self.conv4= nn.Conv2d(128,64,kernel_size=3,stride=1,padding=1)
        self.conv5= nn.Conv2d(64,1,kernel_size=3,stride=1,padding=1)
        
    def forward(self, x,index1,index2,index3,index4):
        x = nnFunctions.max_unpool2d(nnFunctions.relu(self.conv1(x)),indices=index4,kernel_size=(2,2),stride=(2,2))
        x = nnFunctions.relu(self.conv2(x))
        x = nnFunctions.max_unpool2d(nnFunctions.relu(self.conv3(x)),indices=index3,kernel_size=(2,2),stride=(2,2))
        #parallel conv
        x = nnFunctions.max_unpool2d(nnFunctions.relu(self.pconv1(x)+self.pconv2(x)+self.pconv3(x)),indices=index2,kernel_size=(2,2),stride=(2,2))
        
        x = nnFunctions.max_unpool2d(nnFunctions.relu(self.conv4(x)),indices=index1,kernel_size=(2,2),stride=(2,2))
        x = nnFunctions.relu(self.conv5(x))
        return x

In [3]:
vgg16=torchvision.models.vgg16(pretrained=True)

In [4]:
modified_pretrained = nn.Sequential(*list(vgg16.features.children())[:-1])

In [5]:
class finalConvNet(nn.Module):
    global index1,index2,index3,index4
    #constructor
    def __init__(self,mynet,vgg16):
        super(finalConvNet, self).__init__()
        self.vgg=vgg16
        self.custom_net=mynet
        
        for i in range(30):
            if(i<14):
                self.vgg[i].requires_grad=False
            else:
                self.vgg[i].requires_grad=True
            if(i==4):
                self.vgg[i].return_indices=True
            elif(i==9):
                self.vgg[i].return_indices=True
            elif(i==16):
                self.vgg[i].return_indices=True
            elif(i==23):
                self.vgg[i].return_indices=True
        
    def forward(self, x):
        #4 max pool layers
        for layers in range(30):
            if(layers==4):
                x,index1=self.vgg[layers](x)
            elif(layers==9):
                x,index2=self.vgg[layers](x)
            elif(layers==16):
                x,index3=self.vgg[layers](x)
            elif(layers==23):
                x,index4=self.vgg[layers](x)
            else:
                x=self.vgg[layers](x)
        x=self.custom_net(x,index1,index2,index3,index4)
        return x

In [6]:
net=torch.load('Net_checkpoints/net_transfer_learning_5_intermediate5.txt')

In [7]:
net.cuda()

finalConvNet (
  (vgg): Sequential (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU (inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU (inplace)
    (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU (inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU (inplace)
    (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU (inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU (inplace)
    (16): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1)

In [8]:
np_data=np.fromfile('testImagesNumpy_512x512_color.txt')

In [9]:
np_data.shape

(110886912,)

In [10]:
np_data=np_data.reshape(141,512,512,-1)

In [11]:
from numpy import newaxis

In [12]:
np_data=np_data.astype(np.float32)
np_data=np.transpose(np_data,[0,3,1,2])

In [13]:
np_data.shape

(141, 3, 512, 512)

In [14]:
np_labels=np.fromfile('testMaskNumpy_512x512_color.txt')

In [15]:
np_labels.shape

(36962304,)

In [16]:
np_labels=np_labels.reshape(141,512,512,-1)

In [17]:
np_labels=np_labels.astype(np.float32)
np_labels=np.transpose(np_labels,[0,3,1,2])

In [18]:
np_labels.shape

(141, 1, 512, 512)

In [19]:
import torch.utils.data as data_utils

In [20]:
features=torch.from_numpy(np_data)
targets=torch.from_numpy(np_labels)

In [21]:
display = data_utils.TensorDataset(features,targets)
display_loader = data_utils.DataLoader(display, batch_size=4, shuffle=False)

In [22]:
def convertToMask(predicted):
    for i in range(predicted.shape[0]):
        for j in range(predicted.shape[1]):
            for k in range(predicted.shape[2]):
                for l in range(predicted.shape[3]):
                    if(predicted[i][j][k][l]>200):
                        predicted[i][j][k][l]=255
                    else:
                        predicted[i][j][k][l]=0
    return predicted

In [23]:
import matplotlib.pyplot as plt

In [24]:
intersection=0
union=0

In [25]:
for i,data in enumerate(display_loader):
    inputs,mask=data
    # wrap them in Variable
    inputs=Variable(inputs).cuda()
    predicted = net(inputs)
    predicted=predicted.data.cpu().numpy()
    predicted=convertToMask(predicted)
    mask=mask.numpy()
    print ("\r Batch "+str(i)+"Processed",end="")
    sys.stdout.flush()
    intersection+= np.sum( (mask*predicted) > 0 )
    union+=np.sum( (mask+predicted) > 0 )

 Batch 35Processed

In [30]:
print("IoU"+str((intersection)/(1.0*union)))

IoU0.609002965648
